### Download SpeechBrain-MOABB

In [ ]:
%%capture
!pip install --upgrade pip
!git clone https://github.com/speechbrain/benchmarks
%cd /content/benchmarks
!git checkout eeg

!pip install -r /content/benchmarks/benchmarks/MOABB/extra-requirements.txt # Install additional dependencies

### Install the requirements

In [ ]:
%%capture
# Clone SpeechBrain repository (development branch)
%cd /content/
!git clone https://github.com/speechbrain/speechbrain/

# Install required dependencies
!pip install -r /content/speechbrain/requirements.txt

# Install SpeechBrain in editable mode
!pip install -e /content/speechbrain/.

%cd /content/

In [ ]:
%pip install -q mne moabb orion scikit-learn torch torchinfo speechbrain hyperpyyaml braindecode

import mne
import moabb
import orion
import sklearn
import torch
import torchinfo
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml, dump_hyperpyyaml
import braindecode

### Attempt to pre-install packages

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

In [ ]:
# %%capture
# !pip install virtualenv

In [ ]:
# !virtualenv /content/drive/MyDrive/COMP_432/Project/colab_env

In [ ]:
# !source /content/drive/MyDrive/COMP_432/Project/colab_env/bin/activate; pip install mne moabb orion orion[profet] scikit-learn torch torchinfo black==24.3.0 click==8.1.7 flake8==7.0.0 pycodestyle==2.11.0 pydoclint==0.4.1 pytest==7.4.0 yamllint huggingface_hub>=0.8.0 hyperpyyaml>=0.0.1 joblib>=0.14.1 numpy>=1.17.0 packaging pandas>=1.0.1 pre-commit>=2.3.0 scipy>=1.4.1 sentencepiece>=0.1.91 torch>=1.9.0 torchaudio>=1.9.0 tqdm>=4.42.0 transformers>=4.30.0 speechbrain pygtrie>=2.1

In [ ]:
# import sys
# sys.path.append("/content/drive/MyDrive/COMP_432/Project/colab_env/lib/python3.10/site-packages")

In [ ]:
# import mne
# import moabb
# import orion
# import sklearn
# import torch
# import torchinfo
# import speechbrain as sb
# import hyperpyyaml
# from hyperpyyaml import load_hyperpyyaml, dump_hyperpyyaml

In [ ]:
# %%capture
# !git clone https://github.com/speechbrain/benchmarks
# %cd /content/benchmarks
# !git checkout eeg

In [ ]:
# %%capture
# # Clone SpeechBrain repository (development branch)
# %cd /content/
# !git clone https://github.com/speechbrain/speechbrain/

# # # Install SpeechBrain in editable mode
# # !pip install -e /content/speechbrain/.

# %cd /content/

### My own model

In [ ]:
my_model = """
import torch
import speechbrain as sb

class DeepConvNet(torch.nn.Module):
    def __init__(
        self,
        input_shape=None, # (batch_size, time_points, channels(electrodes), num_feature_maps)
        cnn_first_kernels=25,
        cnn_second_kernels=25,
        cnn_third_kernels=50,
        cnn_fourth_kernels=100,
        cnn_fifth_kernels=200,
        cnn_bnorm_momentum=0.1,
        cnn_bnorm_eps=1e-5,
        cnn_max_norm=2,
        cnn_kernelsize=(10, 1),
        cnn_pool_type="max",
        cnn_pool_size=(3, 1),
        dense_n_neurons=4,
        dense_max_norm=0.5,
        dropout=0.4,
        activation_type='elu',
    ):
        super().__init__()
        if input_shape is None:
            raise ValueError("Must specify input_shape")
        if activation_type == "gelu":
            activation = torch.nn.GELU()
        elif activation_type == "elu":
            activation = torch.nn.ELU()
        elif activation_type == "relu":
            activation = torch.nn.ReLU()
        elif activation_type == "leaky_relu":
            activation = torch.nn.LeakyReLU()
        elif activation_type == "prelu":
            activation = torch.nn.PReLU()
        else:
            raise ValueError("Wrong hidden activation function")
        self.default_sf = 128  # sampling rate of the original publication (Hz)
        T = input_shape[1]
        C = input_shape[2]

        #####################################################
        # CONVOLUTIONAL MODULE
        #####################################################
        self.conv_module = torch.nn.Sequential()

        #####################################################
        # 1st conv layer
        self.conv_module.add_module(
            "conv_0",
            sb.nnet.CNN.Conv2d(
                in_channels=1,
                out_channels=cnn_first_kernels,
                kernel_size=cnn_kernelsize,
                padding="valid",
                bias=False,
                max_norm=cnn_max_norm,
                swap=True,
            ),
        )

        #####################################################
        # 2nd conv layer
        self.conv_module.add_module(
            "conv_1",
            sb.nnet.CNN.Conv2d(
                in_channels=cnn_first_kernels,
                out_channels=cnn_second_kernels,
                kernel_size=(1, C),
                padding="valid",
                bias=False,
                max_norm=cnn_max_norm,
                swap=True,
            ),
        )

        # 2nd layer batchnorm
        self.conv_module.add_module(
            "bnorm_1",
            sb.nnet.normalization.BatchNorm2d(
                input_size=cnn_second_kernels,
                momentum=cnn_bnorm_momentum,
                eps=cnn_bnorm_eps,
                affine=True,
            ),
        )

        # 2nd layer activation
        self.conv_module.add_module(
            "act_1",
            activation
        )

        # 2nd layer pooling
        self.conv_module.add_module(
            "pool_1",
            sb.nnet.pooling.Pooling2d(
                pool_type=cnn_pool_type,
                kernel_size=cnn_pool_size,
                stride=cnn_pool_size,
            ),
        )

        # 2nd layer dropout
        self.conv_module.add_module(
            "dropout_1",
            torch.nn.Dropout(p=dropout),
        )

        #####################################################
        # 3rd conv layer
        self.conv_module.add_module(
            "conv_2",
            sb.nnet.CNN.Conv2d(
                in_channels=cnn_second_kernels,
                out_channels=cnn_third_kernels,
                kernel_size=cnn_kernelsize,
                padding="valid",
                bias=False,
                max_norm=cnn_max_norm,
                swap=True,
            ),
        )

        # 3rd layer batchnorm
        self.conv_module.add_module(
            "bnorm_2",
            sb.nnet.normalization.BatchNorm2d(
                input_size=cnn_third_kernels,
                momentum=cnn_bnorm_momentum,
                eps=cnn_bnorm_eps,
                affine=True,
            ),
        )

        # 3rd layer activation
        self.conv_module.add_module(
            "act_2",
            activation
        )

        # 3rd layer pooling
        self.conv_module.add_module(
            "pool_2",
            sb.nnet.pooling.Pooling2d(
                pool_type=cnn_pool_type,
                kernel_size=cnn_pool_size,
                stride=cnn_pool_size,
            ),
        )

        # 3rd layer dropout
        self.conv_module.add_module(
            "dropout_2",
            torch.nn.Dropout(p=dropout),
        )

        #####################################################
        # 4th conv layer
        self.conv_module.add_module(
            "conv_3",
            sb.nnet.CNN.Conv2d(
                in_channels=cnn_third_kernels,
                out_channels=cnn_fourth_kernels,
                kernel_size=cnn_kernelsize,
                padding="valid",
                bias=False,
                max_norm=cnn_max_norm,
                swap=True,
            ),
        )

        # 4th layer batchnorm
        self.conv_module.add_module(
            "bnorm_3",
            sb.nnet.normalization.BatchNorm2d(
                input_size=cnn_fourth_kernels,
                momentum=cnn_bnorm_momentum,
                eps=cnn_bnorm_eps,
                affine=True,
            ),
        )

        # 4th layer activation
        self.conv_module.add_module(
            "act_3",
            activation
        )

        # 4th layer pooling
        self.conv_module.add_module(
            "pool_3",
            sb.nnet.pooling.Pooling2d(
                pool_type=cnn_pool_type,
                kernel_size=cnn_pool_size,
                stride=cnn_pool_size,
            ),
        )

        # 4th layer dropout
        self.conv_module.add_module(
            "dropout_3",
            torch.nn.Dropout(p=dropout),
        )

        #####################################################
        # 5th conv layer
        self.conv_module.add_module(
            "conv_4",
            sb.nnet.CNN.Conv2d(
                in_channels=cnn_fourth_kernels,
                out_channels=cnn_fifth_kernels,
                kernel_size=cnn_kernelsize,
                padding="valid",
                bias=False,
                max_norm=cnn_max_norm,
                swap=True,
            ),
        )

        # 5th layer batchnorm
        self.conv_module.add_module(
            "bnorm_4",
            sb.nnet.normalization.BatchNorm2d(
                input_size=cnn_fifth_kernels,
                momentum=cnn_bnorm_momentum,
                eps=cnn_bnorm_eps,
                affine=True,
            ),
        )

        # 5th layer activation
        self.conv_module.add_module(
            "act_4",
            activation
        )

        # 5th layer pooling
        self.conv_module.add_module(
            "pool_4",
            sb.nnet.pooling.Pooling2d(
                pool_type=cnn_pool_type,
                kernel_size=cnn_pool_size,
                stride=cnn_pool_size,
            ),
        )

        # 5th layer dropout
        self.conv_module.add_module(
            "dropout_4",
            torch.nn.Dropout(p=dropout),
        )

        #####################################################
        # DENSE MODULE
        #####################################################
        # Shape of intermediate feature map
        current_out = self.conv_module(
            torch.ones((1,) + tuple(input_shape[1:-1]) + (1,))
        )
        dense_input_size = self._num_flat_features(current_out)

        self.dense_module = torch.nn.Sequential()

        # flatten
        self.dense_module.add_module(
            "flatten", torch.nn.Flatten(),
        )

        # linear
        self.dense_module.add_module(
            "fc_out",
            sb.nnet.linear.Linear(
                input_size=dense_input_size,
                n_neurons=dense_n_neurons,
                max_norm=dense_max_norm,
            ),
        )

        # final activation
        self.dense_module.add_module("act_out", torch.nn.LogSoftmax(dim=1))

    def _num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    def forward(self, x):
        x = self.conv_module(x)
        x = self.dense_module(x)
        return x
"""

In [ ]:
# save the model into the models folder
f = open('/content/benchmarks/benchmarks/MOABB/models/MyModel.py', "w")
f.write(my_model)
f.close()

### Setting up the yaml file

In [ ]:
tuned_hyperparams = """
# DATASET HPARS
# band-pass filtering cut-off frequencies
fmin: 0.13 # tuned
fmax: 46.0 # tuned
# tmin, tmax respect to stimulus onset that define the interval attribute of the dataset class
# trial begins (0 s), cue (2 s, 1.25 s long); each trial is 6 s long
# dataset interval starts from 2
# -->tmin tmax are referred to this start value (e.g., tmin=0.5 corresponds to 2.5 s)
tmin: 0.
tmax: 4.0 # tuned
# number of steps used when selecting adjacent channels from a seed channel (default at Cz)
n_steps_channel_selection: 2 # tuned

# TRAINING HPARS
# checkpoints to average
avg_models: 10 # tuned
number_of_epochs: 500 # @orion_step1: --number_of_epochs~"uniform(250, 700, discrete=True)"
lr: 0.001 # @orion_step1: --lr~"choices([0.01, 0.005, 0.001, 0.0005, 0.0001])"
batch_size_exponent: 4 # tuned

# DATA AUGMENTATION
# cutcat (disabled when min_num_segments=max_num_segments=1)
max_num_segments: 3 # tuned
cutcat: !new:speechbrain.augment.time_domain.CutCat
    min_num_segments: 2
    max_num_segments: !ref <max_num_segments>
# random amplitude gain between 0.5-1.5 uV (disabled when amp_delta=0.)
amp_delta: 0.01742 # tuned
# random shifts between -300 ms to 300 ms (disabled when shift_delta=0.)
shift_delta_: 1 # tuned
# injection of gaussian white noise
snr_white_low: 15.0 # tuned
snr_white_delta: 19.1 # tuned

# MODEL
cnn_first_kernels: 25 # @orion_step1: --cnn_first_kernels~"uniform(4, 100,discrete=True)"
cnn_second_kernels: ${cnn_first_kernels}
cnn_third_kernels: ${cnn_second_kernels * 2}
cnn_fourth_kernels: ${cnn_third_kernels * 2}
cnn_fifth_kernels: ${cnn_fourth_kernels * 2}
cnn_kernelsize: 10 # @orion_step1: --cnn_kernelsize~"uniform(5, 10,discrete=True)"
cnn_pool_size: 3 # @orion_step1: --cnn_pool_size~"uniform(1, 5,discrete=True)"
dropout: 0.5 # @orion_step1: --dropout~"uniform(0.0, 0.5)"
"""

other_hyperparams = """
seed: 1234
__set_torchseed: !apply:torch.manual_seed [!ref <seed>]

# DIRECTORIES
data_folder: !PLACEHOLDER  #'/path/to/dataset'. The dataset will be automatically downloaded in this folder
cached_data_folder: !PLACEHOLDER #'path/to/pickled/dataset'
output_folder: !PLACEHOLDER #'path/to/results'

# DATASET HPARS
# Defining the MOABB dataset.
dataset: !new:moabb.datasets.BNCI2014001
save_prepared_dataset: True # set to True if you want to save the prepared dataset as a pkl file to load and use afterwards
data_iterator_name: !PLACEHOLDER
target_subject_idx: !PLACEHOLDER
target_session_idx: !PLACEHOLDER
events_to_load: null # all events will be loaded
original_sample_rate: 250 # Original sampling rate provided by dataset authors
sample_rate: 125 # Target sampling rate (Hz)
n_classes: 4
T: !apply:math.ceil
    - !ref <sample_rate> * (<tmax> - <tmin>)
C: 22
# We here specify how to perfom test:
# - If test_with: 'last' we perform test with the latest model.
# - if test_with: 'best, we perform test with the best model (according to the metric specified in test_key)
# The variable avg_models can be used to average the parameters of the last (or best) N saved models before testing.
# This can have a regularization effect. If avg_models: 1, the last (or best) model is used directly.
test_with: 'last' # 'last' or 'best'
test_key: "acc" # Possible opts: "loss", "f1", "auc", "acc"

# METRICS
f1: !name:sklearn.metrics.f1_score
    average: 'macro'
acc: !name:sklearn.metrics.balanced_accuracy_score
cm: !name:sklearn.metrics.confusion_matrix
metrics:
    f1: !ref <f1>
    acc: !ref <acc>
    cm: !ref <cm>

# TRAINING HPARS
n_train_examples: 100  # it will be replaced in the train script
# Learning rate scheduling (cyclic learning rate is used here)
max_lr: !ref <lr> # Upper bound of the cycle (max value of the lr)
base_lr: 0.00000001 # Lower bound in the cycle (min value of the lr)
step_size_multiplier: 5 #from 2 to 8
step_size: !apply:round
    - !ref <step_size_multiplier> * <n_train_examples> / <batch_size>
lr_annealing: !new:speechbrain.nnet.schedulers.CyclicLRScheduler
    base_lr: !ref <base_lr>
    max_lr: !ref <max_lr>
    step_size: !ref <step_size>
label_smoothing: 0.0
loss: !name:speechbrain.nnet.losses.nll_loss
    label_smoothing: !ref <label_smoothing>
optimizer: !name:torch.optim.Adam
    lr: !ref <lr>
epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter  # epoch counter
    limit: !ref <number_of_epochs>
batch_size: !ref 2 ** <batch_size_exponent>
valid_ratio: 0.2

# DATA AUGMENTATION
rand_amp: !new:speechbrain.augment.time_domain.RandAmp
    amp_low: !ref 1 - <amp_delta>
    amp_high: !ref 1 + <amp_delta>
shift_delta: !ref 1e-2 * <shift_delta_> # 0.250 # 0.-0.25 with steps of 0.01
min_shift: !apply:math.floor
    - !ref 0 - <sample_rate> * <shift_delta>
max_shift: !apply:math.floor
    - !ref 0 + <sample_rate> * <shift_delta>
time_shift: !new:speechbrain.augment.freq_domain.RandomShift
    min_shift: !ref <min_shift>
    max_shift: !ref <max_shift>
    dim: 1
snr_white_high: !ref <snr_white_low> + <snr_white_delta>
add_noise_white: !new:speechbrain.augment.time_domain.AddNoise
    snr_low: !ref <snr_white_low>
    snr_high: !ref <snr_white_high>

# pipeline
repeat_augment: 2
augment: !new:speechbrain.augment.augmenter.Augmenter
    parallel_augment: True
    concat_original: True
    parallel_augment_fixed_bs: True
    repeat_augment: !ref <repeat_augment>
    shuffle_augmentations: True
    min_augmentations: 4
    max_augmentations: 4
    augmentations: [
        !ref <cutcat>,
        !ref <rand_amp>,
        !ref <time_shift>,
        !ref <add_noise_white>]

# DATA NORMALIZATION
dims_to_normalize: 1 # 1 (time) or 2 (EEG channels)
normalize: !name:speechbrain.processing.signal_processing.mean_std_norm
    dims: !ref <dims_to_normalize>

# MODEL
input_shape: [null, !ref <T>, !ref <C>, null]
cnn_bnorm_momentum: 0.1
cnn_bnorm_eps: 1e-5
cnn_max_norm: 2
cnn_pool_type: "max"
dense_max_norm: 0.5
activation_type: 'elu'

model: !new:models.MyModel.DeepConvNet
    input_shape: !ref <input_shape>
    cnn_first_kernels: !ref <cnn_first_kernels>
    cnn_second_kernels: !ref <cnn_second_kernels>
    cnn_third_kernels: !ref <cnn_third_kernels>
    cnn_fourth_kernels: !ref <cnn_fourth_kernels>
    cnn_fifth_kernels: !ref <cnn_fifth_kernels>
    cnn_bnorm_momentum: !ref <cnn_bnorm_momentum>
    cnn_bnorm_eps: !ref <cnn_bnorm_eps>
    cnn_max_norm: !ref <cnn_max_norm>
    cnn_kernelsize: [!ref <cnn_kernelsize>, 1]
    cnn_pool_type: !ref <cnn_pool_type>
    cnn_pool_size: [!ref <cnn_pool_size>, 1]
    dense_max_norm: !ref <dense_max_norm>
    dropout: !ref <dropout>
    activation_type: !ref <activation_type>

# # other models that im not using
# model: !new:models.MyModel.EEGNetFusion
#     input_shape: !ref <input_shape>
# model: !new:models.MyModel.MyEEGNet
#     input_shape: !ref <input_shape>
"""

hyperparams = tuned_hyperparams + other_hyperparams

In [ ]:
# Save the yaml file on disk
f = open('/content/hyperparams.yaml', "w")
f.write(hyperparams)
f.close()

### Train the neural network on a single cross-validation fold

In [ ]:
%cd /content/benchmarks/benchmarks/MOABB/

!python train.py /content/hyperparams.yaml \
--data_folder '/content/data/BNCI2014001' \
--cached_data_folder '/content/data' \
--output_folder '/content/results/single-fold-example/BNCI2014001' \
--data_iterator_name 'leave-one-session-out' \
--target_subject_idx 0 \
--target_session_idx 1 \
--number_of_epochs 50 \
--device 'cuda' # Switch to cuda for a speed up.


## Run a complete experiment by looping over the entire dataset



In [ ]:
!./run_experiments.sh --hparams /content/hyperparams.yaml \
--data_folder '/content/data/BNCI2014001'\
--cached_data_folder '/content/data' \
--output_folder '/content/results/full-experiment/BNCI2014001' \
--nsbj 9 --nsess 2 --nruns 1 --train_mode 'leave-one-session-out' \
--number_of_epochs 350 \
--device 'cuda'

## Hyperparameter tuning

In [ ]:
%cd /content/benchmarks/benchmarks/MOABB/

!./run_hparam_optimization.sh --hparams '/content/hyperparams.yaml' \
--data_folder '/content/data/BNCI2014001'\
--cached_data_folder '/content/data' \
--output_folder '/content/results/hyperparameter-search-new/BNCI2014001' \
--nsbj 9 --nsess 2 --nruns 1 --train_mode 'leave-one-session-out' \
--exp_name 'hyperparameter-search' \
--nsbj_hpsearch 1 --nsess_hpsearch 1 \
--nruns_eval 1 \
--eval_metric acc \
--exp_max_trials 5